<a href="https://colab.research.google.com/github/AngelTroncoso/proyecto_telecomx/blob/main/telecoX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Proyecto Telecom2 Preveer que clientes tienen mayor probabilidad de cancelar sus servicios**

🎯 Misión  

Tu nueva misión es desarrollar modelos predictivos capaces de prever qué clientes tienen mayor probabilidad de cancelar sus servicios.  

La empresa quiere anticiparse al problema de la cancelación, y te corresponde a ti construir un pipeline robusto para esta etapa inicial de modelado.

In [83]:
import pandas as pd

In [84]:
url = 'https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'
df = pd.read_json(url)
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [85]:
df = df[df['Churn'].str.strip() != '']

In [86]:
print(df['Churn'].unique())

['No' 'Yes']


In [45]:
df['Churn'].dtype

dtype('O')

In [46]:
df = pd.json_normalize(df.to_dict(orient='records'))
df

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.9
7263,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7
7264,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7265,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


In [47]:
df.shape


(7267, 21)

In [48]:
rename_dict = {
    'customerID': 'ID_cliente',
    'Churn': 'Cancelado',
    'customer.gender': 'Género',
    'customer.SeniorCitizen': 'Adulto_mayor',
    'customer.Partner': 'Tiene_pareja',
    'customer.Dependents': 'Tiene_dependientes',
    'customer.tenure': 'Antigüedad_meses',

    'phone.PhoneService': 'Servicio_telefonía',
    'phone.MultipleLines': 'Líneas_múltiples',

    'internet.InternetService': 'Servicio_internet',
    'internet.OnlineSecurity': 'Seguridad_en_línea',
    'internet.OnlineBackup': 'Respaldo_en_línea',
    'internet.DeviceProtection': 'Protección_dispositivos',
    'internet.TechSupport': 'Soporte_técnico',
    'internet.StreamingTV': 'TV_en_streaming',
    'internet.StreamingMovies': 'Películas_en_streaming',

    'account.Contract': 'Tipo_contrato',
    'account.PaperlessBilling': 'Factura_sin_papel',
    'account.PaymentMethod': 'Método_pago',
    'account.Charges.Monthly': 'Cargo_mensual',
    'account.Charges.Total': 'Cargo_total'
}

In [49]:
df = df.rename(columns=rename_dict)

In [50]:
df.shape

(7267, 21)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID_cliente               7267 non-null   object 
 1   Cancelado                7267 non-null   object 
 2   Género                   7267 non-null   object 
 3   Adulto_mayor             7267 non-null   int64  
 4   Tiene_pareja             7267 non-null   object 
 5   Tiene_dependientes       7267 non-null   object 
 6   Antigüedad_meses         7267 non-null   int64  
 7   Servicio_telefonía       7267 non-null   object 
 8   Líneas_múltiples         7267 non-null   object 
 9   Servicio_internet        7267 non-null   object 
 10  Seguridad_en_línea       7267 non-null   object 
 11  Respaldo_en_línea        7267 non-null   object 
 12  Protección_dispositivos  7267 non-null   object 
 13  Soporte_técnico          7267 non-null   object 
 14  TV_en_streaming         

In [52]:
df.isnull().sum()

,0
ID_cliente,0
Cancelado,0
Género,0
Adulto_mayor,0
Tiene_pareja,0
Tiene_dependientes,0
Antigüedad_meses,0
Servicio_telefonía,0
Líneas_múltiples,0
Servicio_internet,0


In [53]:
df['Cargo_total'] = df['Cargo_total'].str.replace(',', '.', regex=False)

In [54]:
df['Cargo_total'] = pd.to_numeric(df['Cargo_total'], errors='coerce')

In [55]:
df.head()

,ID_cliente,Cancelado,Género,Adulto_mayor,Tiene_pareja,Tiene_dependientes,Antigüedad_meses,Servicio_telefonía,Líneas_múltiples,Servicio_internet,...,Respaldo_en_línea,Protección_dispositivos,Soporte_técnico,TV_en_streaming,Películas_en_streaming,Tipo_contrato,Factura_sin_papel,Método_pago,Cargo_mensual,Cargo_total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40


In [56]:
df.dtypes

,0
ID_cliente,object
Cancelado,object
Género,object
Adulto_mayor,int64
Tiene_pareja,object
Tiene_dependientes,object
Antigüedad_meses,int64
Servicio_telefonía,object
Líneas_múltiples,object
Servicio_internet,object


In [57]:
for col in df.columns:
    print(f"\n📌 Columna: {col}")
    print(df[col].unique())


📌 Columna: ID_cliente
['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']

📌 Columna: Cancelado
['No' 'Yes' '']

📌 Columna: Género
['Female' 'Male']

📌 Columna: Adulto_mayor
[0 1]

📌 Columna: Tiene_pareja
['Yes' 'No']

📌 Columna: Tiene_dependientes
['Yes' 'No']

📌 Columna: Antigüedad_meses
[ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]

📌 Columna: Servicio_telefonía
['Yes' 'No']

📌 Columna: Líneas_múltiples
['No' 'Yes' 'No phone service']

📌 Columna: Servicio_internet
['DSL' 'Fiber optic' 'No']

📌 Columna: Seguridad_en_línea
['No' 'Yes' 'No internet service']

📌 Columna: Respaldo_en_línea
['Yes' 'No' 'No internet service']

📌 Columna: Protección_dispositivos
['No' 'Yes' 'No internet service']

📌 Columna: Soporte_técnico
['Yes' 'No' 'No internet service']

📌 Columna: TV_en_stream

In [58]:
df['Cancelado'].value_counts(dropna=False)

,count
Cancelado,
No,5174
Yes,1869
,224


In [59]:
df['Género'].value_counts(dropna=False)

,count
Género,
Male,3675
Female,3592


In [60]:
#Asumimos 30 días promedio por mes:
df['Cuentas_Diarias'] = df['Cargo_mensual'] / 30

In [61]:
#validación rápida
print(df[['Cargo_mensual', 'Cuentas_Diarias']].head())

   Cargo_mensual  Cuentas_Diarias
0           65.6         2.186667
1           59.9         1.996667
2           73.9         2.463333
3           98.0         3.266667
4           83.9         2.796667


In [62]:
#Resumen estadístico
df['Cuentas_Diarias'].describe()

,Cuentas_Diarias
count,7267.000000
mean,2.157337
std,1.004319
min,0.608333
25%,1.180833
50%,2.343333
75%,2.995833
max,3.958333


In [63]:
df.head()

,ID_cliente,Cancelado,Género,Adulto_mayor,Tiene_pareja,Tiene_dependientes,Antigüedad_meses,Servicio_telefonía,Líneas_múltiples,Servicio_internet,...,Protección_dispositivos,Soporte_técnico,TV_en_streaming,Películas_en_streaming,Tipo_contrato,Factura_sin_papel,Método_pago,Cargo_mensual,Cargo_total,Cuentas_Diarias
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.186667
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,1.996667
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.463333
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,3.266667
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,2.796667


#2.Transformar datos para Machine Learning

In [64]:
#separar datos tipos object y darle una variable de asignacion para extración con cikitlearn
obj_cols = df.select_dtypes(include='object').columns
print("Columnas tipo object:\n", obj_cols)

Columnas tipo object:
 Index(['ID_cliente', 'Cancelado', 'Género', 'Tiene_pareja',
       'Tiene_dependientes', 'Servicio_telefonía', 'Líneas_múltiples',
       'Servicio_internet', 'Seguridad_en_línea', 'Respaldo_en_línea',
       'Protección_dispositivos', 'Soporte_técnico', 'TV_en_streaming',
       'Películas_en_streaming', 'Tipo_contrato', 'Factura_sin_papel',
       'Método_pago'],
      dtype='object')


In [65]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in obj_cols:
    try:
        df[col] = le.fit_transform(df[col])
    except:
        print(f"No se pudo transformar: {col}")

In [66]:
df.nunique()

,0
ID_cliente,7267
Cancelado,3
Género,2
Adulto_mayor,2
Tiene_pareja,2
Tiene_dependientes,2
Antigüedad_meses,73
Servicio_telefonía,2
Líneas_múltiples,3
Servicio_internet,3


In [67]:
df.head()

,ID_cliente,Cancelado,Género,Adulto_mayor,Tiene_pareja,Tiene_dependientes,Antigüedad_meses,Servicio_telefonía,Líneas_múltiples,Servicio_internet,...,Protección_dispositivos,Soporte_técnico,TV_en_streaming,Películas_en_streaming,Tipo_contrato,Factura_sin_papel,Método_pago,Cargo_mensual,Cargo_total,Cuentas_Diarias
0,0,1,0,0,1,1,9,1,0,0,...,0,2,2,0,1,1,3,65.6,593.30,2.186667
1,1,1,1,0,0,0,9,1,2,0,...,0,0,0,2,0,0,3,59.9,542.40,1.996667
2,2,2,1,0,0,0,4,1,0,1,...,2,0,0,0,0,1,2,73.9,280.85,2.463333
3,3,2,1,1,1,0,13,1,0,1,...,2,0,2,2,0,1,2,98.0,1237.85,3.266667
4,4,2,0,1,1,0,3,1,0,1,...,0,2,2,0,0,1,3,83.9,267.40,2.796667


In [68]:
df.describe()

,ID_cliente,Cancelado,Género,Adulto_mayor,Tiene_pareja,Tiene_dependientes,Antigüedad_meses,Servicio_telefonía,Líneas_múltiples,Servicio_internet,...,Protección_dispositivos,Soporte_técnico,TV_en_streaming,Películas_en_streaming,Tipo_contrato,Factura_sin_papel,Método_pago,Cargo_mensual,Cargo_total,Cuentas_Diarias
count,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,...,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7256.000000,7267.000000
mean,3633.000000,1.226366,0.505711,0.162653,0.484106,0.300124,32.346498,0.902711,0.940828,0.875189,...,0.903124,0.796615,0.985414,0.992569,0.688730,0.593230,1.576029,64.720098,2280.634213,2.157337
std,2097.946536,0.486627,0.500002,0.369074,0.499782,0.458343,24.571773,0.296371,0.948332,0.737852,...,0.879297,0.860917,0.884498,0.884587,0.833174,0.491265,1.067676,30.129572,2268.632997,1.004319
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,18.800000,0.608333
25%,1816.500000,1.000000,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,35.425000,400.225000,1.180833
50%,3633.000000,1.000000,1.000000,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,2.000000,70.300000,1391.000000,2.343333
75%,5449.500000,2.000000,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,2.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,89.875000,3785.300000,2.995833
max,7266.000000,2.000000,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,3.000000,118.750000,8684.800000,3.958333


In [69]:
df.dtypes

,0
ID_cliente,int64
Cancelado,int64
Género,int64
Adulto_mayor,int64
Tiene_pareja,int64
Tiene_dependientes,int64
Antigüedad_meses,int64
Servicio_telefonía,int64
Líneas_múltiples,int64
Servicio_internet,int64


In [70]:
df = df.drop("ID_cliente", axis =1)

In [71]:
df

,Cancelado,Género,Adulto_mayor,Tiene_pareja,Tiene_dependientes,Antigüedad_meses,Servicio_telefonía,Líneas_múltiples,Servicio_internet,Seguridad_en_línea,...,Protección_dispositivos,Soporte_técnico,TV_en_streaming,Películas_en_streaming,Tipo_contrato,Factura_sin_papel,Método_pago,Cargo_mensual,Cargo_total,Cuentas_Diarias
0,1,0,0,1,1,9,1,0,0,0,...,0,2,2,0,1,1,3,65.60,593.30,2.186667
1,1,1,0,0,0,9,1,2,0,0,...,0,0,0,2,0,0,3,59.90,542.40,1.996667
2,2,1,0,0,0,4,1,0,1,0,...,2,0,0,0,0,1,2,73.90,280.85,2.463333
3,2,1,1,1,0,13,1,0,1,0,...,2,0,2,2,0,1,2,98.00,1237.85,3.266667
4,2,0,1,1,0,3,1,0,1,0,...,0,2,2,0,0,1,3,83.90,267.40,2.796667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,1,0,0,0,0,13,1,0,0,2,...,0,2,0,0,1,0,3,55.15,742.90,1.838333
7263,2,1,0,1,0,22,1,2,1,0,...,0,0,0,2,0,1,2,85.10,1873.70,2.836667
7264,1,1,0,0,0,2,1,0,0,0,...,0,0,0,0,0,1,3,50.30,92.75,1.676667
7265,1,1,0,1,1,67,1,0,0,2,...,2,2,0,2,2,0,3,67.85,4627.65,2.261667


In [75]:
df = pd.json_normalize(df.to_dict(orient='records'))

# Now the 'Cancelado' column is at the top level and can be accessed directly.
# We can filter out rows where 'Cancelado' is an empty string if necessary.
# Based on the unique values shown later, there are empty strings in 'Cancelado'.
# So, I'll filter those out here.
df = df[df['Cancelado'] != '']
display(df.head())

,Cancelado,Género,Adulto_mayor,Tiene_pareja,Tiene_dependientes,Antigüedad_meses,Servicio_telefonía,Líneas_múltiples,Servicio_internet,Seguridad_en_línea,...,Protección_dispositivos,Soporte_técnico,TV_en_streaming,Películas_en_streaming,Tipo_contrato,Factura_sin_papel,Método_pago,Cargo_mensual,Cargo_total,Cuentas_Diarias
0,1,0,0,1,1,9,1,0,0,0,...,0,2,2,0,1,1,3,65.6,593.30,2.186667
1,1,1,0,0,0,9,1,2,0,0,...,0,0,0,2,0,0,3,59.9,542.40,1.996667
2,2,1,0,0,0,4,1,0,1,0,...,2,0,0,0,0,1,2,73.9,280.85,2.463333
3,2,1,1,1,0,13,1,0,1,0,...,2,0,2,2,0,1,2,98.0,1237.85,3.266667
4,2,0,1,1,0,3,1,0,1,0,...,0,2,2,0,0,1,3,83.9,267.40,2.796667
